In [1]:
import numpy as np
from scipy.stats import ttest_rel
import pandas as pd

In [2]:
store_pvalues = {}
for dataset in ['pereira', 'blank', 'fedorenko']:
    for shuffle_str in ['shuffled', 'contig']:
        for noL2_str in ['_noL2custom', '']:
            results = pd.read_csv(f'/home2/ebrahim/beyond-brainscore/analyze_results/figures_code/figures_data/figure1/{dataset}_pearson_r{noL2_str}_{shuffle_str}.csv')
            for fe in ['', '-mp', '-sp']:
                results_gpt2xl = results.loc[results.Model==f'GPT2-XL{fe}']['perf'].to_numpy()
                results_oasm = results.loc[results.Model=='OASM']['perf'].to_numpy()
                result = ttest_rel(results_gpt2xl, results_oasm)
                if np.mean(results_oasm) > np.mean(results_gpt2xl):
                    store_pvalues[f"{dataset}_{shuffle_str}{noL2_str}_{fe}"] = f"OASM better: {result.pvalue}"
                else:
                    store_pvalues[f"{dataset}_{shuffle_str}{noL2_str}_{fe}"] = f"GPT2XL better: {result.pvalue}"
store_pvalues

{'pereira_shuffled_noL2custom_': 'OASM better: 0.0005829879526030354',
 'pereira_shuffled_noL2custom_-mp': 'OASM better: 8.415461363884617e-06',
 'pereira_shuffled_noL2custom_-sp': 'OASM better: 6.845450682602247e-06',
 'pereira_shuffled_': 'OASM better: 0.0012114104713784794',
 'pereira_shuffled_-mp': 'OASM better: 6.857022634777965e-05',
 'pereira_shuffled_-sp': 'OASM better: 0.00025202230434096434',
 'pereira_contig_noL2custom_': 'GPT2XL better: 0.0006467004109932069',
 'pereira_contig_noL2custom_-mp': 'GPT2XL better: 0.000853685097633342',
 'pereira_contig_noL2custom_-sp': 'GPT2XL better: 0.0009804389292874375',
 'pereira_contig_': 'GPT2XL better: 0.0005507034926934181',
 'pereira_contig_-mp': 'GPT2XL better: 0.00043633780032063205',
 'pereira_contig_-sp': 'GPT2XL better: 0.000749761231008731',
 'blank_shuffled_noL2custom_': 'OASM better: 4.080268110718511e-05',
 'blank_shuffled_noL2custom_-mp': 'OASM better: 5.494779319878125e-05',
 'blank_shuffled_noL2custom_-sp': 'OASM better: 4

In [3]:
store_pvalues = {}
for dataset in ['pereira', 'blank', 'fedorenko']:
    for fe in ['-lt', '-mp', '-sp']:
        results = pd.read_csv(f'/home2/ebrahim/beyond-brainscore/analyze_results/figures_code/figures_data/figure4/{dataset}_pearson_r.csv')
        results_gpt2xl = results.loc[results.Model==f'GPT2XL{fe}']['perf'].to_numpy()
        results_simple = results.loc[results.Model=='Simple']['perf'].to_numpy()
        result = ttest_rel(results_gpt2xl, results_simple)
        
                    
        if np.mean(results_simple) > np.mean(results_gpt2xl):
            store_pvalues[f"{dataset}_{fe}"] = f"Simple better: {result.pvalue}"
        else:
            store_pvalues[f"{dataset}_{fe}"] = f"GPT2XL better: {result.pvalue}"
            
store_pvalues
    

{'pereira_-lt': 'GPT2XL better: 0.004724223518335061',
 'pereira_-mp': 'GPT2XL better: 0.13243890812810272',
 'pereira_-sp': 'GPT2XL better: 0.014500470367014146',
 'blank_-lt': 'Simple better: 0.013885040925368124',
 'blank_-mp': 'Simple better: 0.00904289156248817',
 'blank_-sp': 'Simple better: 0.01127266136623814',
 'fedorenko_-lt': 'GPT2XL better: 0.7785868321863119',
 'fedorenko_-mp': 'GPT2XL better: 0.5708588256662082',
 'fedorenko_-sp': 'GPT2XL better: 0.520582845367957'}

In [ ]:
store_pvalues = {}
for dataset in ['pereira', 'blank', 'fedorenko']:
    for fe in ['-lt', '-mp', '-sp']:
        results = pd.read_csv(f'/home2/ebrahim/beyond-brainscore/analyze_results/figures_code/figures_data/figure5/{dataset}_pearson_r.csv')
        results_gpt2xl = results.loc[results.Model==f'GPT2XL{fe}']['perf'].to_numpy()
        results_simple = results.loc[results.Model=='Simple']['perf'].to_numpy()
        result = ttest_rel(results_gpt2xl, results_simple)
        
                    
        if np.mean(results_simple) > np.mean(results_gpt2xl):
            store_pvalues[f"{dataset}_{fe}"] = f"Simple better: {result.pvalue}"
        else:
            store_pvalues[f"{dataset}_{fe}"] = f"GPT2XL better: {result.pvalue}"
            
store_pvalues

In [19]:
def compute_frac_sig(feature_extraction_arr, dataset, figure_num):
    
    df = pd.read_csv(f'/home2/ebrahim/beyond-brainscore/analyze_results/figures_code/figures_data/figure{figure_num}/pvalues_{dataset}.csv')

    for fe in feature_extraction_arr: 
        
        df_fe = df.loc[df.fe==fe].copy()
        df_fe['pval_sig'] = np.where(df_fe['pval']<0.05, 1, 0)
        df_fe['pval_gpt2xl_sig'] = np.where(df_fe['pval_gpt2xl_sig']<0.05, 1, 0)
        
        
        df_fe = df_fe.loc[df_fe['pval_gpt2xl_sig'] == 1]
        
        if df_fe.shape[0] == 0:
            print(f"No significant voxels/electrodes/fROIs for GPT2XL{fe} for {dataset}")
    
        
        #print(df_fe.groupby('subject')['pval_sig'].sum())

        # Calculate the mean proportion of significant p-values per subject
        subject_means = df_fe.groupby('subject')['pval_sig'].mean()
        
    
        # Calculate the overall mean and standard error
        overall_mean = subject_means.mean() * 100  # Convert to percentage
        
        std_error = (subject_means.std() / np.sqrt(len(subject_means))) * 100  # Convert to percentage

        # Print the results with two significant digits
        print(f"{overall_mean:.3g} {std_error:.3g}%")

In [20]:
compute_frac_sig(['-lt', '-mp', '-sp'], 'pereira',2)
compute_frac_sig(['-lt', '-mp', '-sp'], 'fedorenko',2)
compute_frac_sig(['-lt', '-mp', '-sp'], 'blank',2)

(11173, 9)
21.8 9.1%
(10334, 9)
18.8 7.94%
(10499, 9)
24.1 9.86%
(52, 9)
54.3 10.8%
(52, 9)
54.7 9.25%
(56, 9)
39.5 14.3%
(2, 9)
0 nan%
(22, 9)
0 0%
(17, 9)
0 0%


In [21]:
compute_frac_sig(['-lt', '-mp', '-sp'], 'pereira', 4)
compute_frac_sig(['-lt', '-mp', '-sp'], 'fedorenko', 4)
compute_frac_sig(['-lt', '-mp', '-sp'], 'blank', 4)

(852, 8)
0 0%
(246, 8)
10.7 10.7%
(1116, 8)
6.37 5.72%
(47, 8)
15.6 6.88%
(43, 8)
14.6 8.59%
(48, 8)
12.5 6.18%
(0, 8)
nan nan%
(0, 8)
nan nan%
(0, 8)
nan nan%


In [22]:
compute_frac_sig(['-lt', '-mp', '-sp'], 'pereira',5)
compute_frac_sig(['-lt', '-mp', '-sp'], 'fedorenko',5)
compute_frac_sig(['-lt', '-mp', '-sp'], 'blank',5)

KeyError: 'pval_gpt2xl_sig'